<a href="https://colab.research.google.com/github/ShrutiChrist/Natural-Language-Processing/blob/main/2348545_LAB8%5BNLP%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Weather Prediction

### Loading the data

In [1]:
import pandas as pd
data=pd.read_csv('/content/seattle-weather.csv')
data.head()

date  precipitation  temp_max  temp_min  wind  weather
0  2012-01-01            0.0      12.8       5.0   4.7  drizzle
1  2012-01-02           10.9      10.6       2.8   4.5     rain
2  2012-01-03            0.8      11.7       7.2   2.3     rain
3  2012-01-04           20.3      12.2       5.6   4.7     rain
4  2012-01-05            1.3       8.9       2.8   6.1     rain

### Preprocessing

In [2]:
import numpy as np
import scipy.stats as stats
def drop_outlier(data,col_name):
    z= np.abs(stats.zscore(col_name))
    data.drop(data.index[np.where(abs(z)>2.5)],inplace=True)
drop_outlier(data,data['precipitation'])
drop_outlier(data,data['temp_max'])
drop_outlier(data,data['temp_min'])
drop_outlier(data,data['wind'])
data=data.drop(columns='date')
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
for i in data.drop(columns='weather').columns:
    data[i]=sc.fit_transform(data[[i]])
data.head()

precipitation  temp_max  temp_min      wind  weather
0      -0.503221 -0.540252 -0.679922  1.265942  drizzle
1       2.239635 -0.840696 -1.123174  1.108110     rain
2      -0.301911 -0.690474 -0.236670 -0.628046     rain
4      -0.176092 -1.072858 -1.123174  2.370769     rain
5       0.125874 -1.687403 -1.244060 -0.706962     rain

### Train and Test Split

In [3]:
X=data.drop(columns='weather')
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
Y=le.fit_transform(data['weather'])
from tensorflow.keras.utils import to_categorical
Y= to_categorical(Y)
from sklearn.model_selection import train_test_split
X_train,X_val,Y_train,Y_val=train_test_split(X,Y,stratify=Y,test_size=0.2)

### LSTM model

In [4]:
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, Input
from tensorflow.keras.models import Sequential
model=Sequential()
model.add(Input(shape=(X_train.shape[1],1)))
model.add(LSTM(32, activation='relu',return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, activation='relu',return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

In [5]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 32)             4352      
                                                                 
 dropout (Dropout)           (None, 4, 32)             0         
                                                                 
 lstm_1 (LSTM)               (None, 4, 64)             24832     
                                                                 
 dropout_1 (Dropout)         (None, 4, 64)             0         
                                                                 
 lstm_2 (LSTM)               (None, 128)               98816     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 5)                 6

In [ ]:
history=model.fit(X_train, Y_train, batch_size=32, epochs=50,validation_data=(X_val, Y_val))

Epoch 1/50
35/35 [==============================] - 1s 15ms/step - loss: 0.4759 - accuracy: 0.8458 - val_loss: 0.5640 - val_accuracy: 0.8285
Epoch 2/50
35/35 [==============================] - 1s 15ms/step - loss: 0.4847 - accuracy: 0.8476 - val_loss: 0.5553 - val_accuracy: 0.8248
Epoch 3/50
35/35 [==============================] - 1s 18ms/step - loss: 0.4814 - accuracy: 0.8458 - val_loss: 0.5499 - val_accuracy: 0.8248
Epoch 4/50
35/35 [==============================] - 1s 14ms/step - loss: 0.4903 - accuracy: 0.8422 - val_loss: 0.5550 - val_accuracy: 0.8248
Epoch 5/50
35/35 [==============================] - 0s 14ms/step - loss: 0.4864 - accuracy: 0.8431 - val_loss: 0.5616 - val_accuracy: 0.8175
Epoch 6/50
35/35 [==============================] - 0s 14ms/step - loss: 0.4753 - accuracy: 0.8467 - val_loss: 0.5580 - val_accuracy: 0.8285
Epoch 7/50
35/35 [==============================] - 0s 13ms/step - loss: 0.4681 - accuracy: 0.8476 - val_loss: 0.5564 - val_accuracy: 0.8321
Epoch 8/50
35

### Training and testing loss and accuracy

In [8]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(50)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Machine Translation

### Statistical machine translation (SMT) using googletrans

In [9]:
!pip install googletrans==3.1.0a0


In [10]:
import nltk
from googletrans import Translator

In [11]:
def translate_text(text, source_lang, dest_lang):
    translator = Translator()
    translated = translator.translate(text, src=source_lang, dest=dest_lang)
    return translated.text

In [12]:
input_text = "Hello, how are you?"
translated_text = translate_text(input_text,'english','kannada')
print("Translated Text:", translated_text)

Translated Text: ಹಲೋ, ಹೇಗಿದ್ದೀಯಾ?
